In [2]:
import os
import re
import cv2
import numpy as np
import warnings

In [3]:
def new_folder(dir_name, verbose=False):
    """
    Check whether given directory exists and create new one if not.
    
    Parameters:
        dir_name: string
            Name of the directory to be checked and to be created.
        verbose: bool (default False)
            Whether or not to inform the user about the creation of the folder.
    """
    # Input management
    if not isinstance(dir_name, str):
        raise ValueError("Different datatype than string has been given as input for name of the directory.")

    if not isinstance(verbose, bool):
        raise ValueError("Different datatype than boolean has been given as input for the verbose parameter")

    # Check directory's existence and try to create it if not existing and if possible
    if not os.path.exists(dir_name):
        try:
            os.makedirs(dir_name)
            if verbose:
                print(f"A folder has been successfully created with this path: {dir_name}")
        except OSError:
            wrn = "\nThe path you have specified for the new folder to-be created is invalid."
            wrn += "\nThe folder does not exist and was not created. Please specify a correct path."
            warnings.warn(wrn)

    elif verbose:
        print(f"A folder with this name already exists: {dir_name}")

In [4]:
def repair_padding(dir_name):
    """
    Images padded as A_1.jpg, A_2.jpg, ... are expected for the project.
    This function detects skips in padding (e.g. A_1.jpg, A_3.jpg) and repairs these.

    Parameters:
        dir_name: str
            Path to the directory that needs repair.
    """
    # Input management
    if not isinstance(dir_name, str):
        raise ValueError("Different datatype than string has been given as input for name of the directory.")

    if not os.path.exists(dir_name):
        warnings.warn("\nDirectory with given path does not exist. No padding adjustments have been made, returning.")
        return

    # Obtain the list of files in the folder
    files = os.listdir(dir_name)
    files.sort(key=lambda file: int(re.split(r"[_|.]", file)[1]))
    filecount = len(files)

    # The first image should have 1 in it's padding, rename it if necessary
    if filecount:
        first_split = re.split(r"[_|.]", files[0])
        if int(first_split[1]) != 1:
            new_name = first_split[0] + "_" + str(1) + "." + first_split[2]
            os.rename(os.path.join(dir_name, files[0]), os.path.join(dir_name, new_name))
            files = os.listdir(dir_name)
            files.sort(key=lambda file: int(re.split(r"[_|.]", file)[1]))
        
    # Go through each file and if the run order skips a count, shift all the following files' padding
    for i in range(filecount - 1):
        name_split = re.split(r"[_|.]", files[i])
        name_split_next = re.split(r"[_|.]", files[i + 1])

        # Stop the process if there is some unexpected (i.e. < 1) number in padding
        if int(name_split_next[1]) < 1:
            wrn = f"\nIn the folder {dir_name} there is a file {files[i + 1]} with non-positive numbering!\n"
            wrn += "This might result in unexpected behaviour. The repair process will now terminate, please correct the number."
            warnings.warn(wrn)
            return

        # Warn the user if there is some mix of gestures
        if name_split[0] != name_split_next[0]:
            warnings.warn(f"There are files with different naming in the current directory ({dir_name}): {files[i]}, {files[i + 1]}!")

        # Find the padding jump
        diff = int(name_split_next[1]) - int(name_split[1])

        # The expected padding difference is one, 
        # otherwise something is missing and all following files need to be shifted
        if diff != 1:
            for j in range(i + 1, filecount):
                name_current = re.split(r"[_|.]", files[j])
                new_name = name_current[0] + "_" + str(int(name_current[1]) - diff + 1) + "." + name_current[2]
                os.rename(os.path.join(dir_name, files[j]), os.path.join(dir_name, new_name))

            # Get the new list of files with adjusted padding
            files = os.listdir(dir_name)
            files.sort(key=lambda file: int(re.split(r"[_|.]", file)[1]))

In [5]:
def setup_folders(script_directory, gestures_list, amount_per_gesture):
    """
    Setup folders for the project. Add directories for data and examples
    and subfolders for each gesture in the list of gestures. Also setup
    the current and desired amounts for each gesture.
    
    Parameters:
        script_directory: str
            Name of the directory where the script is located. All folders
            will be created in this directory.
        gestures_list: list of strings
            List of all gestures to be included in the project. 
            Subfolder will be created for each one in the Data folder.
            Example should be present for each one in Examples folder.
        amount_per_gesture: int
            Positive integer that specifies the desired number of datapoints per gesture.

    Returns:
        data_directory: str
            Directory name for the Data folder. Subfolders for each gesture are located in this folder.
        example_directory: str
            Directory name of the Examples folder.
            If not present beforehand, it is created and filled with dummy images per gesture.
        desired_amount: dict of str: int pairs
            Dictionary of desired amounts of samples per each gesture.
        current_amount: dict of str: int pairs
            Dictionary of current amounts of samples per each gesture.
        paths: dict of str: str
            Dictionary of paths to all the gestures.
    """
    # Input management
    if not isinstance(script_directory, str):
        raise ValueError("Different datatype than string has been given as input for name of the script directory.")
    else:
        if not script_directory:
            warnings.warn("Given script directory is empty, thus all the processes will run in the location of this script.")
        else:
            if not os.path.exists(script_directory):
                raise ValueError("The given directory for the script does not exist, please specify a correct directory path.")

    if not isinstance(gestures_list, list):
        raise ValueError("Different datatype than list has been given as input for the list of gestures.")
    else:
        for val in gestures_list:
            if not isinstance(val, str):
                raise ValueError("There is a value with different datatype than string in the list of gestures.")
        if len(gestures_list) != len(set(gestures_list)):
            raise ValueError("There are some gesture duplicates in the list of gestures, please check.")

    if not isinstance(amount_per_gesture, int):
        raise ValueError("Different datatype than integer has been given as input for the desired amount per gesture.")

    if amount_per_gesture < 0:
        raise ValueError("Negative number has been given as input for the desired amount per gesture.")

    # Specify the desired number of images for each gesture
    desired_amount = {gesture: amount_per_gesture for gesture in gestures_list}

    # Initialize the dictionary of current number of occurrences per each gesture
    current_amount = {gesture: 0 for gesture in gestures_list}

    # Initialize folder names, initialize paths dictionary
    data_dir = os.path.join(script_directory, "Data")
    example_dir = os.path.join(script_directory, "Examples")
    paths = {}

    # Create Data and Example folders if they do not not exist yet
    new_folder(data_dir, verbose=True)
    new_folder(example_dir, verbose=True)

    for gesture in gestures_list:

        # Create a subfolder per each gesture if it does not exist yet and add it to paths
        new = os.path.join(data_dir, gesture)
        new_folder(new)
        paths[gesture] = new

        # If the subfolder exists, make sure that the ordering is correct and
        # shift it if any skips are present
        # (e.g. "A_1.jpg", "A_2.jpg", ... instead of "A_1.jpg", "A_3.jpg", ...)
        repair_padding(new)

        # Since the directory is now correctly sorted by padding, we can read the current amounts
        files = os.listdir(new)
        files.sort(key=lambda file: int(re.split(r"[_|.]", file)[1]))
        current_amount[gesture] = 0 if not files else int(re.split(r"[_|.]", files[-1])[1])

        # Create a dummy example image for the current gesture if it does not have an example image yet
        new = os.path.join(example_dir, gesture + ".jpg")
        if not os.path.exists(new):
            cv2.imwrite(f"{new}", np.ones((540, 960)) * 255)
            warnings.warn(f"\nThe current gesture ({gesture}) does not have an example image yet, a dummy image has been created instead.")

    return data_dir, example_dir, desired_amount, current_amount, paths

In [6]:
def create_rectangle(origin, size_X, size_Y):
    """
    Function to create the rectangle shape with given properties
    
    Parameters:
        origin: tuple of int, int
            Upper left corner of the rectangle.
        size_X: int
            Width of the rectangle
        size_Y: int
            Height of the rectangle

    Returns:
        rectangle: list of 4 tuples of (int, int)
            The coordinates for the upper left, upper right, lower left and lower right corner
    """
    if not isinstance(origin, tuple):
        raise ValueError("Different datatype than tuple has been given as input for the origin.")
    elif len(origin) != 2:
        raise ValueError("Different dimension than 2 given for the origin of the rectangle.")
    else:
        for val in origin:
            if not isinstance(val, int):
                raise ValueError("Different datatype than integer has been given for the coordinates of the origin.")
            elif val < 0:
                raise ValueError("The coordinates for the origin cannot be negative.")

    if not isinstance(size_X, int) or not isinstance(size_Y, int):
        raise ValueError("The height and width of the rectangle must be integers.")
    elif size_X < 0 or size_Y < 0:
        raise ValueError("The height and width of the rectangle must be positive integers.")

    X, Y = origin
    upper_right, lower_left, lower_right = (X + size_X, Y), (X, Y + size_Y), (X + size_X, Y + size_Y)
    return [origin, upper_right, lower_left, lower_right]

In [22]:
def image_capturing(gesture_list, examples="Examples", save=True, predict=False, 
                    data_directory=None, current_amounts=None, desired_amounts=None, 
                    gesture_paths=None, model=None):
    """
    Function for image capturing. Enables usage as data collector or purely as image obtainer.
    If the save parameter is True, thresholded binary images in the rectangle you see on the screen
    will be saved in appropriate directory for each gesture.
    
    Throughout the run of the function, you can use the following commands by using keys on your keyboard:
        Esc - terminate the whole process
        q - skip the current gesture and move to the next one
        l - switch language from English to Czech and vice versa
        spacebar - move the rectangle into the other position (one should be more comfortable for fingerspelling)
    
    Parameters:
        gesture_list: list of str
            List of gestures to go through
        examples: str (default "Examples")
            Name of the folder with examples.
        save: bool (default True)
            Whether or not to save the captured images, i.e. whether or not to collect new data.
            If True, legitimate directory for saving needs to be specified.
        predict: bool (default False)
            Whether or not to include prediction on the screen
        data_directory: str
            Name of the folder in which to save the images in case save is True.
        current_amounts: dict of str: int pairs
            Dictionary of current amounts of datapoints per each gesture. Only relevant in case save is True.
        desired_amounts: dict of str: int pairs
            Dictionary of desired amounts of datapoints per each gesture. Only relevant in case save is True.
        gesture_paths: dict of str: str pairs
            Dictionary of paths for each gesture. Only relevant in case save is True.
        model: keras.engine.sequential.Sequential
            Model that the user would like to use for prediction.
    """
    # Input management
    if not isinstance(gesture_list, list):
        raise ValueError("Different datatype than list has been given as input for the list of gestures.")
    else:
        for val in gesture_list:
            if not isinstance(val, str):
                raise ValueError("Different datatype than string has been given for one of the values in the list of gestures.")
        if len(gesture_list) != len(set(gesture_list)):
            raise ValueError("There are some gesture duplicates in the list of gestures, please check.")

    if not isinstance(examples, str):
        raise ValueError("Different datatype than string has been given for the name of the folder with examples.")
    else:
        if not os.path.exists(examples):
            raise ValueError("The given directory for the examples does not exist, please specify a correct directory path.")
        else:
            files = os.listdir(examples)
            if len(files) != len(gesture_list):
                raise ValueError("The length of the example folder does not correspond to the list of gestures, please adjust.")
            gesture_names = [re.split(r"[.]", file)[0] for file in files]
            if len(gesture_names) != len(set(gesture_names)):
                raise ValueError("There are some gesture duplicates in the example folder, please check.")
            if set(gesture_names) != set(gesture_list):
                raise ValueError("The list of gestures does not correspond to the files in the given example folder.")

    if not isinstance(save, bool):
        raise ValueError("Different datatype than boolean has been given as input for the save parameter.")

    if save:
        pass
        # Deploy further input management for other parameters
        # TODO !!!! 

    # The rectangle in the frame that is cropped from the web camera image 
    # (one for torso location, one for fingerspelling location)
    rect_torso = create_rectangle((225, 275), 200, 200)
    rect_fingerspell_1 = create_rectangle((50, 50), 200, 200)
    rect_fingerspell_2 = create_rectangle((400, 50), 200, 200)
    rect = rect_torso

    cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
    
    # Encapsulate the whole process to be able to close cameras in case of error
    try:

        # Establish the windows and place them accordingly
        cv2.namedWindow("Camera view")
        cv2.resizeWindow("Camera view", 640, 480)
        cv2.moveWindow("Camera view", 15, 200)

        cv2.namedWindow("Grayscale view")
        cv2.resizeWindow("Grayscale view", 480, 360)
        cv2.moveWindow("Grayscale view", 655, 30)

        cv2.namedWindow("Binary view")
        cv2.resizeWindow("Binary view", 480, 360)
        cv2.moveWindow("Binary view", 655, 430)

        cv2.namedWindow("Example")
        cv2.resizeWindow("Example", 380, 270)
        cv2.moveWindow("Example", 1125, 280)

        if predict:
            gestures_folder = os.listdir(data_directory)

        lang = True  # To let the user change language, True stands for English, False for Czech
        rectangle_position = 0  # Which position of the rectangle to use

        # Perform the data collecting process for each gesture in the given gesture list
        for gesture in gesture_list:

            # Initialize necessary variables (different per gesture)
            # There is no limit on when to end during the process of not saving - has to be commanded manually
            if save:
                current = current_amounts[gesture] + 1
                end = desired_amounts[gesture]
            else:
                current = 0
                end = np.inf
            counter = current
            
            flag = 0  # To know when a new gesture is being taken for the first time
            exit = 0  # To let the user end the process early by clicking the "Esc" key

            # Continue until the respective subfolder has the designated number of samples
            while counter <= end:
                ret, frame = cap.read()

                # Check validity and avoid mirroring if frame is present
                if not ret:
                    print("There has been a problem retrieving your frame")
                    break
                else:
                    frame = cv2.flip(frame, 1)

                # End the process for the current gesture in case the "q" key is hit
                key = cv2.waitKey(1)
                if key == ord("q"):
                    break

                # End the whole process in case the "Esc" key is hit
                if key == ord("\x1b"):
                    exit = 1
                    break

                # Change language settings in case the "l" key is hit
                if key == ord("l"):
                    lang = not lang

                # Change the rectangle position if the "spacebar" key is hit
                if key == ord(" "):
                    rectangle_position += 1
                    rect = [rect_torso, rect_fingerspell_1, rect_fingerspell_2][rectangle_position % 3]

                # Create grayscale version
                frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

                # Binarize the grayscale image using adaptive thresholding
                frame_binary = frame_gray[(rect[0][1] + 2):(rect[2][1] - 2), 
                                          (rect[0][0] + 2):(rect[1][0] - 2)]

                # Preprocessing, denoising, blurring
                frame_binary = cv2.fastNlMeansDenoising(frame_binary, None, 5, 15, 7)
                frame_binary = cv2.medianBlur(frame_binary, 3)
                frame_binary = cv2.GaussianBlur(frame_binary, (3, 3), 0)
                # Adaptive thresholding
                frame_binary = cv2.adaptiveThreshold(frame_binary, 255, cv2.ADAPTIVE_THRESH_MEAN_C, 
                                                     cv2.THRESH_BINARY_INV, 3, 2)
                # Closing operation on the thresholded image
                kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
                frame_binary = cv2.morphologyEx(frame_binary, cv2.MORPH_CLOSE, kernel)

                # Show all images
                
                # Live view with frame and text
                cv2.rectangle(frame, rect[0], rect[3], (0, 255, 0), 2)
                # Add information about prediction if expected, otherwise just show the name of the gesture
                if predict:
                    prediction = model(np.expand_dims(np.expand_dims(cv2.resize(frame_binary, 
                                                                                (64, 64)), 
                                                                     axis=0), axis=-1), 
                                       training=False).numpy()
                    txt = gestures_folder[np.argmax(prediction, axis=1)[0]]
                else:
                    txt = gesture.capitalize()
                cv2.putText(frame, txt, (rect[0][0], rect[0][1] - 15), 
                            cv2.FONT_HERSHEY_DUPLEX, 1, (0, 255, 0), 2)
                cv2.imshow("Camera view", frame)

                # Grayscale version
                cv2.imshow("Grayscale view", cv2.resize(frame_gray, (480, 360)))

                # Binary version
                cv2.imshow("Binary view", cv2.resize(frame_binary, (480, 360)))

                # Show example on new gesture
                if not flag:
                    example = cv2.imread(f"{os.path.join(examples, gesture)}" + ".jpg")
                    cv2.imshow("Example", cv2.resize(example, (380, 270)))
                    flag = 1

                # To reduce the number of almost identical frames, only save every n frames
                # To give space for adjustments and "learning" a new sign, only start collecting after some time
                if save:
                    if not current % 2 and current > 90:

                        # Create the naming for the file with the desired padding, i.e. ("gesture_run-number.jpg")
                        img_name = gesture + "_" + str(counter) + ".jpg"
                        img_path = r"%s" % os.path.join(gesture_paths[gesture], img_name)

                        # Save the cropped rectangle from the frame
                        if not cv2.imwrite(img_path, 
                                           cv2.resize(frame_binary, (64, 64))):
                            print("Something went wrong during this attempt:",
                                  f"gesture - {gesture}, run - {counter}")

                        counter += 1

                current += 1

            if exit:
                break

    # Close the camera and all windows in case of unexpected fatality
    finally:
        cap.release()
        cv2.destroyAllWindows()
        return

In [8]:
# Specify the list of gestures, a subfolder will be created for each one
gestures = ["I index", "My", "You", "Your",
            "In", "To", "With", "Yes",
            "No", "Well", "I love you", "Oh I see",
            "Name", "Hug", "Internet", "Bus",
            "Money", "Work", "Ask",
            "Go", "Look", "Have", "Correct",
            "Want", "Where", 
            "A", "B", "C", "D",
            "E", "F", "G", "H",
            "I", "K", "L", "M",
            "N", "O", "P", "Q",
            "R", "S", "T", "U",
            "V", "W", "X", "Y"]

script_dir = os.path.dirname("Image Collection.ipynb")

# Specify and set up the folder environment, amounts and paths for all the gestures in the gesture list
data_dir, example_dir, desired_amount, current_amount, paths = setup_folders(script_dir, gestures, 500)

<ipython-input-5-8b264c977d17>:36: UserWarning: Given script directory is empty, thus all the processes will run in the location of this script.
  warnings.warn("Given script directory is empty, thus all the processes will run in the location of this script.")


A folder with this name already exists: Data
A folder with this name already exists: Examples


In [9]:
import CNN

In [10]:
model = CNN.build_model(labels=49, convolutional_layers=4, input_shape=(64, 64))
model.load_weights("Weights/weights")

In [11]:
import numpy as np
import cv2

im = cv2.imread("Data/Yes/Yes_88.jpg", cv2.IMREAD_GRAYSCALE)
im = np.expand_dims(im, axis=0)
im = np.expand_dims(im, axis=-1)
prediction = model(im, training=False)
os.listdir("Data")[np.argmax(prediction, axis = 1)[0]]

'Yes'

In [21]:
# To test model collection, uncomment and run the lines below 
# Note that the "desired_amount" per gesture might be satisfied, thus the process will not run
# In this case, re-run the above setup_folders function but raise the number of samples per gesture (now 500)

#image_capturing(gestures, save=True, data_directory=data_dir, current_amounts=current_amount,
#                desired_amounts=desired_amount, gesture_paths=paths)

##################################################################

# To test model showcasing, uncomment and run the lines below

#image_capturing(gestures, save=False, predict=True, 
#                    data_directory=data_dir, current_amounts=current_amount, desired_amounts=desired_amount, 
#                    gesture_paths=paths, model=model)

##################################################################

# To test data collection script showcasing (no model prediction, no saving), uncomment and run the line below
#image_capturing(gestures, save=False)

In [13]:
# List of some morphological operations I have experimented with (with no real success)

#kernel = np.ones((5, 5), np.uint8)
#kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
#frame_binary = cv2.morphologyEx(frame_binary, cv2.MORPH_GRADIENT, kernel)
#frame_binary = cv2.erode(frame_binary, kernel, iterations=1)
#frame_binary = cv2.dilate(frame_binary, kernel, iterations=1)
#frame_binary = cv2.medianBlur(frame_binary, 3)
#frame_binary = cv2.threshold(frame_gray, 150, 255, cv2.THRESH_BINARY)[1]
#frame_binary = cv2.erode(frame_binary, kernel, iterations=3)
#frame_binary = cv2.dilate(frame_binary, kernel, iterations=2)
#frame_binary = cv2.morphologyEx(frame_binary, cv2.MORPH_GRADIENT, kernel)

In [14]:
# Code to find contours in the binarized frame and add them to the original frame

# Find contours and draw them into the original image
#contours = cv2.findContours(frame_binary.copy(), 
#                            cv2.RETR_EXTERNAL, 
#                            cv2.CHAIN_APPROX_SIMPLE)[0]
#if contours:
#    cv2.drawContours(frame, 
#                     max(contours, key=cv2.contourArea) + rect[0], 
#                     -1, 
#                     (0, 0, 255), 3)

In [15]:
# Old version of the function including two methods for thresholding

old = """

def image_capturing(method="adaptive", binary_threshold=0, last_mask=-1):
    ""Capture image from the camera, convert it to grayscale 
    and then perform preprocessing and thresholding.
    
    Keyword arguments:
    method (string) -- hand segmentation method selection from (default adaptive)
        mask = use background substraction based on mask taken at the beginning
        adaptive = use adaptive thresholding methods
    binary_thresh (integer) -- the thresholding parameter for binary thresholding in mask method (default 150)
    last_mask (integer) -- the last frame to consider for the creation of background mask in mask method
    ""
    # Input management
    if method not in ["adaptive", "mask"]:
        raise ValueError("Method other than adaptive or mask has been given as input")
    
    if not isinstance(binary_threshold, int):
        raise ValueError("Different datatype than integer has been given as input for binary threshold")
    
    if not isinstance(last_mask, int):
        raise ValueError("Different datatype than integer has been given as input for last frame of background mask")
    
    if method == "mask" and binary_threshold == 0:
        wrn = "\nMask thresholding method has been chosen but binary threshold has not been specified."
        wrn += "\nThis will result in a fully one-colored image, please specify the threshold"
        warnings.warn(wrn)

    if method == "mask" and (binary_threshold < 0 or binary_threshold >= 255):
        wrn = "\nMask thresholding method has been chosen but binary threshold is under 0 or over 255."
        wrn += "\nThis will result in a fully one-colored image, please specify different threshold (between 1 and 254)"
        warnings.warn(wrn)

    if method == "mask" and last_mask <= 10:
        if last_mask == -1:
            wrn = "\nBackground substraction thresholding method has been chosen but last mask frame has not been specified."
        else:
            wrn = "\nBackground substraction thresholding method has been chosen but last mask frame is very low (<= 10)."
        wrn += "\nThis may result in unpredictable behaviour and possibly crash of the whole script, please correct the parameter"
        warnings.warn(wrn)

    # The rectangle in the frame that is cropped from the web camera image
    rect = [(225, 275), (425, 275), 
            (225, 475), (425, 475)]

    cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

    # Encapsulate the whole process to be able to close cameras in case of error
    try:

        # Establish the windows and place them accordingly
        cv2.namedWindow("Camera view")
        cv2.resizeWindow("Camera view", 640, 480)
        cv2.moveWindow("Camera view", 15, 200)

        cv2.namedWindow("Grayscale view")
        cv2.resizeWindow("Grayscale view", 480, 360)
        cv2.moveWindow("Grayscale view", 655, 30)

        cv2.namedWindow("Binary view")
        cv2.resizeWindow("Binary view", 480, 360)
        cv2.moveWindow("Binary view", 655, 430)

        cv2.namedWindow("Example")
        cv2.resizeWindow("Example", 380, 270)
        cv2.moveWindow("Example", 1125, 280)

        # Initialize variables for background substraction
        frame_count = 0
        background = None

        # Perform the data collecting process for each gesture in the given gesture list
        for gesture in gestures:

            # Initialize necessary variables (different per gesture)
            current = current_amount[gesture] + 1
            counter = current
            end = desired_amount[gesture]
            flag = 0 # To know when a new gesture is being taken for the first time
            exit = 0 # To let the user end the process early by clicking the "Esc" key

            # Continue until the respective subfolder has the designated number of samples
            while counter <= end:
                ret, frame = cap.read()

                # Check validity and avoid mirroring if frame is present
                if not ret:
                    print("There has been a problem retrieving your frame")
                    break
                else:
                    frame_count += 1
                    frame = cv2.flip(frame, 1)

                # End the process for the current gesture in case the "q" key is hit
                key = cv2.waitKey(1)
                if key == ord("q"):
                    break

                # End the whole process in case the "Esc" key is hit
                if key == ord("\x1b"):
                    exit = 1
                    break

                # Create grayscale version
                frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

                # Binarize the grayscale image using thresholding (with various methods)
                frame_binary = frame_gray[(rect[0][1] + 2):(rect[2][1] - 2), 
                                          (rect[0][0] + 2):(rect[1][0] - 2)]
                
                # Version using background mask
                if method == "mask":
                    # Create mask for the background
                    if background is None:
                        background = frame_binary.copy().astype("float")
                    else:
                        if frame_count <= last_mask:
                            background = cv2.accumulateWeighted(frame_binary, background, 0.5)
                        # Use the mask for background substraction
                        else:
                            frame_binary = cv2.absdiff(background.astype("uint8"), frame_binary)
                            frame_binary = cv2.GaussianBlur(frame_binary, (3, 3), 0)
                            frame_binary = cv2.threshold(frame_binary, binary_threshold, 255, cv2.THRESH_BINARY)[1]

                # Version using adaptive thresholding
                elif method == "adaptive":
                    # Preprocessing, denoising, blurring
                    frame_binary = cv2.fastNlMeansDenoising(frame_binary, None, 5, 15, 7)
                    frame_binary = cv2.medianBlur(frame_binary, 3)
                    frame_binary = cv2.GaussianBlur(frame_binary, (3, 3), 0)
                    # Adaptive thresholding
                    frame_binary = cv2.adaptiveThreshold(frame_binary, 255, cv2.ADAPTIVE_THRESH_MEAN_C, 
                                                         cv2.THRESH_BINARY_INV, 3, 2)
                    # Closing operation on the thresholded image
                    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
                    frame_binary = cv2.morphologyEx(frame_binary, cv2.MORPH_CLOSE, kernel)

                # Show all images
                
                # Live view with frame and text
                cv2.rectangle(frame, rect[0], rect[3], (0, 255, 0), 2)
                #acc = 0.0
                #txt = gesture.capitalize() + f" ({str(round(acc, 2))} %)"      # in preparation for model version
                txt = gesture.capitalize()
                cv2.putText(frame, txt, (rect[0][0], rect[0][1] - 15), 
                            cv2.FONT_HERSHEY_DUPLEX, 1, (0, 255, 0), 2)
                cv2.imshow("Camera view", frame)

                # Grayscale version
                cv2.imshow("Grayscale view", cv2.resize(frame_gray, (480, 360)))

                # Binary version
                if frame_count > last_mask:
                    cv2.imshow("Binary view", cv2.resize(frame_binary, (480, 360)))

                # Show example on new gesture
                if not flag:
                    example = cv2.imread(f"{os.path.join(example_dir, gesture)}" + ".jpg")
                    cv2.imshow("Example", cv2.resize(example, (380, 270)))
                    #time.sleep(1)
                    flag = 1

                # To reduce the number of almost identical frames, only save every n frames
                # To give space for adjustments and "learning" a new sign, only start collecting after some time
                if not current % 2 and current > 90:

                    # Create the naming for the file with the desired padding, i.e. ("gesture_run-number.jpg")
                    img_name = gesture + "_" + str(counter) + ".jpg"
                    img_path = r"%s" %os.path.join(paths[gesture], img_name)

                    # Save the cropped rectangle from the frame
                    if not cv2.imwrite(img_path, 
                                       frame_binary):
                        print("Something went wrong during this attempt:",
                              f"gesture - {gesture}, run - {counter}")

                    counter += 1

                current += 1

            if exit:
                break

        cap.release()
        cv2.destroyAllWindows()
        return

    # Close the camera and all windows in case of unexpected fatality
    except:
        print("A fatality has occured, the program will now terminate")
        cap.release()
        cv2.destroyAllWindows()
        return

"""